In [17]:
import time

### Input data paths:

# Gioia

In [36]:
dataset_path = "../../Bio_Project/SimDataset/reads_datasets/all_250000_1.fq"
#classifier_path = "../../Bio_Project/SimDataset/classifiers_results/strex_centrifuge_250000.res"
#classifier_path = "../../Bio_Project/SimDataset/classifiers_results/strex_kraken2_250000.res"
classifier_path = "../../Bio_Project/SimDataset/classifiers_results/strex_clark_genus_250000.res"
clusters_path =  "../../Bio_Project/results/LiME_binning_all_250000_1/all_250000_1+RC.fasta.a16.t20.txt"

IsFasta = False

# Leo

In [ ]:
#dataset_path = "../all_118309_1.fq"
#clusters_path =  "../all_118309_1+RC.fasta.a16.t20.txt"
#classifier_path = "../strex_kraken1_118309.res"

dataset_path = "../all_250000_1.fq"
clusters_path =  "../all_250000_1+RC.fasta.a16.t20.txt"
#classifier_path = "../strex_kraken2_250000.res"
classifier_path = "../strex_centrifuge_250000.res"




IsFasta = False

#### Load dataset function 
- If dataset is in fasta format then dataset_format parameter equals TRUE.
- If dataset is in fastq format then dataset_format equals FALSE.



In [19]:
def load_dataset(path, datset_format : bool):

    dataset = open(path, "r")
    dataset_lines = []

    if (datset_format):
        divisor = 2
    else:
        divisor = 4

    index = 0
    for line in dataset:
        if (index%divisor==0):
            read_id = line.split()[0]
            dataset_lines.append(read_id[1: len(read_id)-2])
        index = index + 1

    dataset.close()

    return dataset_lines

#### Load clusters result function

In [20]:
def load_clusters_result(path):

    clusters = open(path, "r")
    clusters_list = []

    for group in clusters:
        clusters_list.append(int(group))

    clusters.close()

    return clusters_list

#### Load classifier result function

In [21]:
def load_classifier_result(path):

    classification = open(path, 'r')
    classifier_results = {}

    for line in classification:
        col = []
        for j in range(0, len(line.split())):
            col.append(line.split()[j])
        #read_id = line.split()[0]
        #class_id = line.split()[1]
        #col.append(read_id)
        #col.append(class_id)
        
        classifier_results[col[0]] = col[1]

    classification.close()

    return classifier_results 

#### Get inverted index function

In [22]:
def get_inverted_index(clusters, read_ids):

    num_clusters = max(clusters) + 1
    num_reads = len(read_ids)
    inverted_index = []
    
    for i in range(0, num_clusters):
        inverted_index.append([])
   
    #try to append in order
    for i in range(0, num_reads):
        inverted_index[read_ids[i][2]].append(read_ids[i][1])

    return inverted_index

#### Find labels function

In [23]:
def find_labels(read_ids, classification_output):

    complete_classifier_result = []
    found : bool = False
    count = 0
    for i in range(0, len(read_ids)):
        for j in range(0, len(classification_output)):
            col = []
            if (read_ids[i] == classification_output[j][0]):
                complete_classifier_result.append(classification_output[j])
                found = True
                break
        if not found:
            count = count +1
            complete_classifier_result.append(read_ids[i], '0')

    print("count: ", count)
    print("length classification output: ", len(classification_output))
    print("length complete classification output: ", len(complete_classifier_result))

    return complete_classifier_result

#### Find labels frequency function
This function takes in input a cluster (group of reads, each read is identified by its read_id) and the classification of reads performed by a classifier
and return a dictionary composed by key-value pairs where the key is a label and value equals to the frequency of such label in the cluster provided in input



In [24]:
#ricorda gestione non presente --> classe 0
# dict --> (classe, volte che la vedo nel cluster)

def frequency_search_binary(cluster):
    
    label_dict= {}
       
    for read in cluster:         
    
        if read in list(label_dict):
            label_dict[read] = label_dict[read] + 1
        else:
            # if it does not exist, it's automatically created
            label_dict[read] = 1
        
    return label_dict

#### Find lonely reads function

In [25]:
def find_lonely_reads(clusters):

    lonely_reads = []

    for i in range(0, len(clusters)):
        if (len(clusters[i]) == 1):
            lonely_reads.append(clusters[i][0])

    return lonely_reads

#### Find non classified reads function

In [26]:
def find_non_classified_reads(read_ids, classification_output):

    non_classified_reads = []

    if (len(read_ids) > len(classification_output)):
        print("le read non classificate non sono presenti nel file di output, DA IMPLEMENTARE")
    elif (len(read_ids) == len(classification_output)):
        ###print("le read non classificate hanno label = 0")
        for i in range(0, len(classification_output)):
            if (classification_output[i][1] == '0'):
                non_classified_reads.append(classification_output[i][0])
    else:
        print("Error in input files!")

    return non_classified_reads

#### Basic operation functions
- intersection
- binary search
- insertion sort
- diff
- binary search list
- insertion sort list

In [27]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3


# Iterative Binary Search Function
# It returns index of x in given array arr if present,
# else returns -1
def binary_search(arr, x):
    low = 0
    high = len(arr) - 1
    mid = 0
    while low <= high:
        mid = (high + low) // 2
        # If x is greater, ignore left half
        if arr[mid] < x:
            low = mid + 1
        # If x is smaller, ignore right half
        elif arr[mid] > x:
            high = mid - 1
        # means x is present at mid
        else:
            return mid
    # If we reach here, then the element was not present
    return -1


def insertion_sort(arr):
        
    for i in range(len(arr)):
        cursor = arr[i]
        pos = i  
        while pos > 0 and arr[pos - 1] > cursor:
            # Swap the number down the list
            arr[pos] = arr[pos - 1]
            pos = pos - 1
        # Break and do the final swap
        arr[pos] = cursor
    return arr

# Python code t get difference of two lists
# Not using set()
def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif

# Iterative Binary Search Function
# It returns index of x in given array arr if present,
# else returns -1
def binary_search_list(arr, x):
    low = 0
    high = len(arr) - 1
    mid = 0
    while low <= high:
        mid = (high + low) // 2
        #print(arr[mid][0])
        # If x is greater, ignore left half
        if arr[mid][0] < x:
            low = mid + 1
        # If x is smaller, ignore right half
        elif arr[mid][0] > x:
            high = mid - 1
        # means x is present at mid
        else:
            return mid
    # If we reach here, then the element was not present
    return -1



def insertion_sort_list(arr):
        
    for i in range(len(arr)):
        cursor = arr[i][0]
        pos = i
        while pos > 0 and arr[pos - 1][0] > cursor:
            # Swap the number down the list
            arr[pos][0] = arr[pos - 1][0]
            pos = pos - 1
        # Break and do the final swap
        arr[pos][0] = cursor

    return arr



#### Find labels (improved) function

In [28]:
def find_labels_improved(read_ids, classification_output):

    complete_classifier_result = []
    found : bool = False
    count = 0

    classification_output = insertion_sort(classification_output)

    print("classification_output sorted")

    for i in range(0, len(read_ids)):
        position = binary_search_list(classification_output, str(read_ids[i]))
        if (position == -1):
            count = count + 1
            complete_classifier_result.append([read_ids[i], '0'])
        else:
            complete_classifier_result.append(classification_output[position])

    print("count: ", count)
    print("length classification output: ", len(classification_output))
    print("length complete classification output: ", len(complete_classifier_result))

    return complete_classifier_result

#### Reassignment function

In [29]:
def reassignment(dataset, max_labels):

    reassigned_classification = []

    for i in range(0, len(dataset)):
        cluster_index = dataset[i][2]
        reassigned_classification.append([dataset[i][0], max_labels[cluster_index]])
                                          
    return reassigned_classification

In [30]:
def build_dataset(dataset_ids,clusters,classifier):
    final_dataset =[]
        
    for i in range(0, len(dataset_ids)):
        final_dataset.append([])

        if dataset_ids[i] in classifier:
            final_dataset[i].append(dataset_ids[i])
            final_dataset[i].append(classifier[dataset_ids[i]])
            final_dataset[i].append(clusters[i])
        else:
            final_dataset[i].append(dataset_ids[i])
            final_dataset[i].append(0)
            final_dataset[i].append(clusters[i])
        
    return final_dataset

## Main code

In [37]:
start = time.time()


dataset_lines = load_dataset(dataset_path, IsFasta)
clusters_list = load_clusters_result(clusters_path)
classifier_results = load_classifier_result(classifier_path)

#new Structure : id , classifier result , cluster

dataset = build_dataset(dataset_lines,clusters_list,classifier_results)

stop = time.time()

print("Time for loading and creating datasets: ", stop-start)



if (len(dataset_lines) == len(clusters_list)):
    num_reads = len(dataset_lines)
else:
    print ("Error in input files!")
    exit()


start = time.time()

# INVERTED INDEX
inverted_index = get_inverted_index(clusters_list, dataset)

stop = time.time()

print("Time for getting inverted index: ", stop-start)
    
    
start = time.time()
i = 0
max_label_per_cluster_list = []
max_label_list = []
for cluster in inverted_index:
    label_dict = frequency_search_binary(cluster)
    max_label = ""
    max_count = 0
    for label, count in label_dict.items():
        #print(label, count)
        if (count > max_count):
            max_count = count
            max_label = label
    #print("CLUSTER: ", i, " MAX_LABEL: ", max_label, "MAX COUNT: ", max_count, "CLUSTER LENGTH: ", len(cluster))
    max_label_list.append(max_label)
    max_label_per_cluster_list.append([max_label, max_count, len(cluster)])
    i = i + 1

    
reassigned_classification = reassignment(dataset, max_label_list) 


stop = time.time()

print("Time for class definition: ", stop-start)


start = time.time()

stop = time.time()

print("Time for class definition: ", stop-start)

Time for loading and creating datasets:  25.532742023468018
Time for getting inverted index:  1.7932662963867188
Time for class definition:  10.827752590179443
Time for class definition:  0.0


### Saving results

In [40]:
f = open("ClassificationReassignedNotOrdered.res", "w")

for element in reassigned_classification:
    f.write(element[0])
    f.write("\t")
    f.write(str(element[1]))
    f.write("\n")
f.close()